In [1]:
from datasets import load_dataset

# Load private dataset
#dataset = load_dataset("mikeriess/LM_Da_test_100_llama-3.3-70b-instruct")
dataset = load_dataset("mikeriess/LM_Da_test_100_llama-3.1-405b-instruct-fp8")

/Users/mikeriess/anaconda3/envs/LMLLM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 18/18 [00:00<00:00, 1891.50 examples/s]


In [2]:
# Convert to pandas and then to dictionary format
df = dataset['train'].to_pandas()
df

,question_id,model_generator,conversation,language,turn
0,c7b456b374984591bd10663833fc29e4,meta-llama/Llama-3.1-405B-Instruct-FP8,"[{'content': 'Jeg har hørt, at det ikke er mul...",Danish,1
1,c7b456b374984591bd10663833fc29e4,meta-llama/Llama-3.1-405B-Instruct-FP8,[{'content': 'Det lyder godt. Men hvordan gør ...,Danish,2
2,c7b456b374984591bd10663833fc29e4,meta-llama/Llama-3.1-405B-Instruct-FP8,"[{'content': 'Ah, okay. Det havde jeg ikke set...",Danish,3
3,a870d08355694e87948fc84f85078953,meta-llama/Llama-3.1-405B-Instruct-FP8,"[{'content': 'Hej, jeg sidder og kæmper med Lo...",Danish,1
4,a870d08355694e87948fc84f85078953,meta-llama/Llama-3.1-405B-Instruct-FP8,"[{'content': 'Ah, tak! Det er rigtig nyttigt a...",Danish,2
5,a870d08355694e87948fc84f85078953,meta-llama/Llama-3.1-405B-Instruct-FP8,[{'content': 'Perfekt! Og kan jeg også få dem ...,Danish,3
6,07efa9b392084b4699615e018592a2a3,meta-llama/Llama-3.1-405B-Instruct-FP8,"[{'content': 'Har du set den nye video, som mi...",Danish,1
7,07efa9b392084b4699615e018592a2a3,meta-llama/Llama-3.1-405B-Instruct-FP8,"[{'content': 'Ja, jeg synes selv den er ret mo...",Danish,2
8,07efa9b392084b4699615e018592a2a3,meta-llama/Llama-3.1-405B-Instruct-FP8,"[{'content': 'Ja, hun har altid været god til ...",Danish,3
9,39b79bdde296462eab626eec1d2318dd,meta-llama/Llama-3.1-405B-Instruct-FP8,[{'content': 'Hvordan får jeg en muted subgrup...,Danish,1


In [3]:
df.loc[df.question_id == "a870d08355694e87948fc84f85078953"]['conversation']

3    [{'content': 'Hej, jeg sidder og kæmper med Lo...
4    [{'content': 'Ah, tak! Det er rigtig nyttigt a...
5    [{'content': 'Perfekt! Og kan jeg også få dem ...
Name: conversation, dtype: object

In [4]:
# Convert dataset to pandas
df = dataset['train'].to_pandas()

# Group by question_id and concatenate conversations
chatml_data = {}
for question_id in df['question_id'].unique():
    # Get all rows for this question_id
    question_rows = df[df['question_id'] == question_id]
    
    # Sort by order if needed
    question_rows = question_rows.sort_values('turn')
    
    # Create messages list
    messages = []
    for _, row in question_rows.iterrows():
        # Assuming each conversation entry is a list of dicts with 'content' key
        for message in row['conversation']:
            messages.append({
                "role": "user" if len(messages) % 2 == 0 else "assistant",
                "content": message['content']
            })
    
    # Store in chatml format
    chatml_data[question_id] = {
        "messages": messages,
        "language": question_rows.iloc[0]['language']  # Take language from first row
    }

chatml_data

{'c7b456b374984591bd10663833fc29e4': {'messages': [{'role': 'user',
    'content': 'Jeg har hørt, at det ikke er muligt at lave en Autoload i Logic 8. Er det rigtigt?'},
   {'role': 'assistant',
    'content': 'Nej, det er ikke rigtigt. Du kan lave dine egne templates og indstille Logic til at loade en af dem automatisk ved opstart.'},
   {'role': 'user',
    'content': 'Det lyder godt. Men hvordan gør jeg det? Jeg har allerede prøvet at lave en template og kalde den Autoload, men det virker ikke.'},
   {'role': 'assistant',
    'content': 'Ah, det er ikke sådan, du gør det. Du skal gå ind i preferences og sætte den ønskede template som standardtemplate ved opstart.'},
   {'role': 'user',
    'content': 'Ah, okay. Det havde jeg ikke set. Og hvad med mine gamle Logic 7 Autoloads? Skal jeg slette dem, eller kan de stadig bruges?'},
   {'role': 'assistant',
    'content': 'Det er bedst at slette dem, da de kan give problemer på visse setups, især på Intel Macs. Og selv om de ikke gør det,

In [5]:

print("Sample conversation:")
sample_id = list(chatml_data.keys())[1]
print(f"\nQuestion ID: {sample_id}")
print(f"Language: {chatml_data[sample_id]['language']}")
print("\nMessages:")
for msg in chatml_data[sample_id]['messages']:
    #print(f"{msg['role']}: {msg['content'][:100]}...")
    print(f"{msg['role']}: {msg['content']}"+ "\n")

Sample conversation:

Question ID: a870d08355694e87948fc84f85078953
Language: Danish

Messages:
user: Hej, jeg sidder og kæmper med Logic 8. Er det muligt at få vist både pianoroll og score på samme tid?

assistant: Ja, det er faktisk muligt. Du kan åbne både "Piano Roll Editor" og "Score Editor" samtidig ved at holde "Shift" ned mens du åbner den anden editor.

user: Ah, tak! Det er rigtig nyttigt at vide. Men hvordan får jeg dem så til at opdatere sig selv, hvis jeg laver ændringer i den ene?

assistant: Det er også ret enkelt. Du kan bare aktivere "Link" funktionen i toppen af editorerne. På den måde vil ændringer i den ene editor automatisk blive overført til den anden.

user: Perfekt! Og kan jeg også få dem til at vise forskellige dele af projektet på samme tid?

assistant: Ja, du kan bruge "Split View" funktionen til at vise forskellige dele af projektet i hver editor. Du kan for eksempel vise pianorollen for et instrument i den ene editor, og partituret for hele projektet i den 